### Loading data

In [1]:
def load_dataset():
    # We first define a download function, supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve

    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)

    # We then define functions for loading MNIST images and labels.
    # For convenience, they also download the requested files if needed.
    import gzip

    def load_mnist_images(filename):
        #if not os.path.exists(filename):
        #    download(filename)
        # Read the inputs in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        # The inputs are vectors now, we reshape them to monochrome 2D images,
        # following the shape convention: (examples, channels, rows, columns)
        data = data.reshape(-1, 1, 784)
        # The inputs come as bytes, we convert them to float32 in range [0,1].
        # (Actually to range [0, 255/256], for compatibility to the version
        # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
        return data / np.float32(256)

    def load_mnist_labels(filename):
        #if not os.path.exists(filename):
        #    download(filename)
        # Read the labels in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        # The labels are vectors of integers now, that's exactly what we want.
        return data

    # We can now download and read the training and test set images and labels.
    script_dir=os.getcwd()
    rel_path="Input_data/"
    abs_path=os.path.join(script_dir,rel_path)
    X_train = load_mnist_images(abs_path+'train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels(abs_path+'train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images(abs_path+'t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels(abs_path+'t10k-labels-idx1-ubyte.gz')

    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_val, y_val, X_test, y_test

### Success rate on MNIST dataset without PCA

In [2]:
import numpy as np
import sys
import os

from matplotlib import pyplot as plt

%matplotlib inline

X_train, y_train, X_val, y_val, X_test, y_test=load_dataset()

X_train=X_train.reshape((50000,784))

from sklearn import svm

clf=svm.LinearSVC()

clf.fit(X_train,y_train)

X_val=X_val.reshape((10000,784))

clf.score(X_val,y_val)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


0.92069999999999996

In [ ]:
clf.score(X_train,y_train)

In [ ]:
X_test=X_test.reshape((10000,784))
clf.score(X_test,y_test)

In [ ]:
clf.coef_.shape

In [ ]:
for i in range(10):
    print np.linalg.norm(clf.coef_[i])

In [ ]:
clf.decision_function(X_train)

In [ ]:
one_array=np.where(y_train==1)

# one_array[0]

one_y_train=y_train[one_array]

# one_y_train.shape

one_X_train=X_train[one_array[0],:]

# one_X_train.shape

one_decision_train=clf.decision_function(one_X_train)

# one_decision_train[0,:]

one_correct=np.where(one_decision_train<0)

# one_correct



# np.amax(one_decision_train[:,1])

# np.amin(np.abs(one_decision_train[:,1]))

positive_indices_one=np.where(one_decision_train[:,1]>0)[0]

temp_gap=np.where(one_decision_train[:,1]>1.1)[0]

# temp_gap.shape

# np.amin(one_decision_train[positive_indices_one,1])

In [ ]:
clf_one=svm.LinearSVC()
clf_.fit(X_train,y_train)

In [ ]:
clf.score(one_X_train,y_train[one_array])

In [ ]:
positive_indices_one=np.where(one_decision_train[:,1]>0)[0]

In [ ]:
np.amin(one_decision_train[positive_indices_one,1]/np.linalg.norm(clf.coef_[1,:]))

In [ ]:
np.amax(one_decision_train[positive_indices_one,1]/np.linalg.norm(clf.coef_[1,:]))

In [ ]:
np.mean(one_decision_train[positive_indices_one,1]/np.linalg.norm(clf.coef_[1,:]))

### Finding adversarial example success rates

In [3]:
no_of_mags=10
adv_examples_train=np.zeros((50000,784))

mag_count=0
for DEV_MAG in np.linspace(1.0,1.0,1):
    count_pure_adv=0.0
    count_adv=0.0
    count_wrong=0.0
    count_correct=0.0
    print_flag=0
    for i in range(50000):
        x_ini=(X_train[i,:]).reshape((1,784))
        ini_class=clf.predict(x_ini)
        x_adv=(x_ini-DEV_MAG*(clf.coef_[ini_class[0],:]/(np.linalg.norm(clf.coef_[ini_class[0],:])))).reshape((1,784))
        x_adv=x_adv
        adv_examples_train[i,:]=x_adv
        final_class=clf.predict(x_adv)
        if ini_class[0]==y_train[i]:
            count_correct=count_correct+1
        if ini_class[0]!=final_class[0]:
            count_adv=count_adv+1
        if y_train[i]!=final_class[0]:
            count_wrong=count_wrong+1
        if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
            count_pure_adv=count_pure_adv+1
#         if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#             #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#             plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#             print_flag=print_flag+1
    mag_count=mag_count+1
print count_wrong/50000.0, count_adv/50000.0, count_pure_adv/count_correct

0.95928 0.99976 0.9997408767


In [4]:
X_train_inc_adv=np.vstack((X_train,adv_examples_train))
y_train_new=np.hstack((y_train,y_train))

In [5]:
clf_new=svm.LinearSVC()
clf_new.fit(X_train_inc_adv,y_train_new)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [7]:
X_test=X_test.reshape((10000,784))
clf_new.score(X_test,y_test)

0.91549999999999998

a=3.0*(clf.coef_[ini_class[0],:]/(np.linalg.norm(clf.coef_[ini_class[0],:])))

np.amax(a)

a.shape

np.linalg.norm(a,1)

In [9]:
no_of_mags=10
adv_examples_test=np.zeros((10000,784,no_of_mags))

mag_count=0
for DEV_MAG in np.linspace(0.1,1.0,10):
    count_pure_adv=0.0
    count_adv=0.0
    count_wrong=0.0
    count_correct=0.0
    print_flag=0
    for i in range(10000):
        x_ini=(X_test[i,:]).reshape((1,784))
        ini_class=clf_new.predict(x_ini)
        x_adv=(x_ini-DEV_MAG*(clf_new.coef_[ini_class[0],:]/(np.linalg.norm(clf_new.coef_[ini_class[0],:])))).reshape((1,784))
        adv_examples_test[i,:,mag_count]=x_adv
        final_class=clf_new.predict(x_adv)
        if ini_class[0]==y_test[i]:
            count_correct=count_correct+1
        if ini_class[0]!=final_class[0]:
            count_adv=count_adv+1
        if y_test[i]!=final_class[0]:
            count_wrong=count_wrong+1
        if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
            count_pure_adv=count_pure_adv+1
    #     if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
    #         fig=plt.imshow((x_adv.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
    #         plt.savefig('MNIST_SVM_'+str(2)+'_'+str(ini_class)+'_'+str(final_class)+'.png',bbox_inches='tight')
    #         fig=plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
    #         plt.savefig('MNIST_SVM_'+str(ini_class)+'.png',bbox_inches='tight')
            #fig=plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
    mag_count=mag_count+1
    print count_wrong/10000.0, count_adv/10000.0, count_pure_adv/count_correct
    print count_correct

0.2114 0.2399 0.185253959585
9155.0
0.5352 0.5824 0.5453850355
9155.0
0.8089 0.8579 0.844893500819
9155.0
0.9099 0.9581 0.954341889678
9155.0
0.9428 0.9905 0.989623156745
9155.0
0.9513 0.9985 0.998361551065
9155.0
0.9541 1.0 1.0
9155.0
0.9552 1.0 1.0
9155.0
0.9558 1.0 1.0
9155.0
0.9573 1.0 1.0
9155.0


In [ ]:
final_class[0]

In [ ]:
plt.imshow((X_test[13,:].reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

In [ ]:
ini_class

In [ ]:
final_class

In [ ]:
a=clf.decision_function(x_ini)[0][6]/np.linalg.norm(clf.coef_[ini_class[0],:])

In [ ]:
b=clf.decision_function(x_adv)[0][6]/np.linalg.norm(clf.coef_[ini_class[0],:])

In [ ]:
a-b

In [ ]:
one_decision_adv=clf.decision_function(adv_examples_train_one)

In [ ]:
np.amax(one_decision_adv[:,1])

In [ ]:
np.amin(one_decision_adv[:,1])

In [ ]:
temp_gap_adv=np.where(one_decision_adv[:,1]>0)[0]

In [ ]:
temp_gap_adv

In [ ]:
for i in range(10):
    print 1/np.linalg.norm(clf.coef_[i,:])

### Success rate on MNIST dataset with PCA

In [ ]:
from sklearn.decomposition import PCA

rd=50

PCA_in_train=X_train
PCA_in_val=X_val
PCA_in_test=X_test

### Doing PCA over the training data
#Fitting the PCA model on training data
pca=PCA(n_components=rd)
pca.fit(PCA_in_train)
#Transforming the training, validation and test data
X_train_dr=pca.transform(PCA_in_train).reshape((50000,rd))
X_test_dr=pca.transform(PCA_in_test).reshape((10000,rd))
X_val_dr=pca.transform(PCA_in_val).reshape((10000,rd))

clf_pca=svm.LinearSVC(dual=False)

clf_pca.fit(X_train_dr,y_train)

# val_out_pca=clf_pca.predict(X_val_dr)
# validation_success_pca=(10000-np.count_nonzero(val_out_pca-y_val))/10000.0

In [ ]:
X_test_rev=pca.inverse_transform(X_test_dr)
X_train_rev=pca.inverse_transform(X_train_dr)

In [ ]:
clf_pca.score(X_train_dr,y_train)

In [ ]:
clf_pca.score(X_test_dr,y_test)

In [ ]:
one_X_train_dr=X_train_dr[one_array[0],:]

clf_pca.score(one_X_train_dr,one_y_train)

one_decision_train_dr=clf_pca.decision_function(one_X_train_dr)

np.amax(one_decision_train_dr[:,1])

np.amin(one_decision_train_dr[:,1])

temp_gap_dr=np.where(one_decision_train_dr[:,1]>1.1)[0]

temp_gap_dr.shape

In [ ]:
positive_indices_one_dr=np.where(one_decision_train_dr[:,1]>0)[0]

In [ ]:
np.amax(one_decision_train_dr[positive_indices_one_dr,1]/np.linalg.norm(clf_pca.coef_[1,:]))

In [ ]:
np.amin(one_decision_train_dr[positive_indices_one_dr,1]/np.linalg.norm(clf_pca.coef_[1,:]))

In [ ]:
np.mean(one_decision_train_dr[positive_indices_one_dr,1]/np.linalg.norm(clf_pca.coef_[1,:]))

In [ ]:
for i in range(10):
    print np.linalg.norm(clf_pca.coef_[i,:])

In [ ]:
X_adv_test_dr=pca.transform(adv_examples_test).reshape((10000,rd))
rev_adv_test=pca.inverse_transform(X_adv_test_dr)
X_adv_train_dr=pca.transform(adv_examples_train).reshape((50000,rd))
rev_adv_train=pca.inverse_transform(X_adv_train_dr)

In [ ]:
X_adv_test_dr[13,:].shape

In [ ]:
plt.imshow((X_adv_test_dr[13,:].reshape((1,rd)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
plt.axis('off')
plt.savefig('rd_input.png',bbox_inches='tight')

In [ ]:
rev_train=pca.inverse_transform(X_train_dr)
clf.score(rev_train,y_train)

In [ ]:
rev_test=pca.inverse_transform(X_test_dr)
clf.score(rev_test,y_test)

### Reconstruction based defense

In [ ]:
count_pure_adv_2=0.0
count_adv_2=0.0
count_wrong_2=0.0
count_correct=0.0
for i in range(10000):
    x_ini_2=(X_test_rev[i,:]).reshape((1,784))
    ini_class_2=clf.predict(x_ini_2)
    #x_adv=(x_ini-1.0*(clf.coef_[ini_class[0],:]/(np.linalg.norm(clf.coef_[ini_class[0],:])))).reshape((1,784))
    x_adv_2=rev_adv_test[i,:].reshape((1,784))
    final_class_2=clf.predict(x_adv_2)
    if ini_class_2[0]!=final_class_2[0]:
        count_adv_2=count_adv_2+1
    if y_test[i]!=final_class_2[0]:
        count_wrong_2=count_wrong_2+1
    if y_test[i]==ini_class_2[0]:
        count_correct=count_correct+1
    if y_test[i]!=final_class_2[0] and ini_class_2[0]==y_test[i]:
        count_pure_adv_2=count_pure_adv_2+1
#     plt.imshow((adv_examples_test[0].reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#     if ini_class_2[0]==y_test[i]:
#         fig=plt.imshow((x_adv_2.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.savefig('MNIST_SVM_recons'+str(rd)+'_'+str(2)+'_'+str(ini_class_2)+'_'+str(final_class_2)+'.png',bbox_inches='tight')
#         fig=plt.imshow((x_ini_2.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.savefig('MNIST_SVM_recons'+str(rd)+'_'+str(ini_class_2)+'.png',bbox_inches='tight')
print count_wrong_2/10000.0, count_adv_2/10000.0, count_pure_adv_2/count_correct
print count_correct

plt.imshow((rev_adv_test[0].reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

In [ ]:
count_pure_adv_2=0.0
count_adv_2=0.0
count_wrong_2=0.0
count_correct=0.0
for i in range(50000):
    x_ini_2=(X_train_rev[i,:]).reshape((1,784))
    ini_class_2=clf.predict(x_ini_2)
    #x_adv=(x_ini-1.0*(clf.coef_[ini_class[0],:]/(np.linalg.norm(clf.coef_[ini_class[0],:])))).reshape((1,784))
    x_adv_2=rev_adv_train[i,:].reshape((1,784))
    final_class_2=clf.predict(x_adv_2)
    if ini_class_2[0]!=final_class_2[0]:
        count_adv_2=count_adv_2+1
    if y_train[i]!=final_class_2[0]:
        count_wrong_2=count_wrong_2+1
    if y_train[i]==ini_class_2[0]:
        count_correct=count_correct+1
    if y_train[i]!=final_class_2[0] and ini_class_2[0]==y_train[i]:
        count_pure_adv_2=count_pure_adv_2+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong_2/50000.0, count_adv_2/50000.0, count_pure_adv_2/count_correct
print count_correct

In [ ]:
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(50000):
    x_ini=(X_train_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
#     x_adv=(x_ini-DEV_MAG*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    x_adv=X_adv_train_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if y_train[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if y_train[i]==ini_class[0]:
        count_correct=count_correct+1
    if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/50000.0, count_adv_pca/50000.0, count_pure_adv_pca/count_correct
print count_correct

In [ ]:
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(10000):
    x_ini=(X_test_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
#     x_adv=(x_ini-DEV_MAG*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    x_adv=X_adv_test_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if y_test[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if y_test[i]==ini_class[0]:
        count_correct=count_correct+1
    if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/10000.0, count_adv_pca/10000.0, count_pure_adv_pca/count_correct
print count_correct

### Retraining-based, keeping same average distortion per pixel

In [ ]:
DEV_MAG=1.0
DEV_MAG_rd=DEV_MAG*np.sqrt(50.0/784.0)
DEV_MAG_rd

In [ ]:
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(50000):
    x_ini=(X_train_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
    x_adv=(x_ini-DEV_MAG*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    #x_adv=X_adv_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if y_train[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if y_train[i]==ini_class[0]:
        count_correct=count_correct+1
    if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/50000.0, count_adv_pca/50000.0, count_pure_adv_pca/count_correct
print count_correct

In [ ]:
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(10000):
    x_ini=(X_test_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
    x_adv=(x_ini-DEV_MAG*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    #x_adv=X_adv_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if y_test[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if y_test[i]==ini_class[0]:
        count_correct=count_correct+1
    if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/10000.0, count_adv_pca/10000.0, count_pure_adv_pca/count_correct
print count_correct

In [ ]:
one_X_adv_dr=np.zeros((5678,rd))
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
#print_flag_pca=0
for i in range(5678):
    x_ini=(one_X_train_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
    x_adv=(x_ini-1.0*(clf_pca.coef_[1,:]/(np.linalg.norm(clf_pca.coef_[1,:])))).reshape((1,rd))
    one_X_adv_dr[i,:]=x_adv
    #x_adv=X_adv_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if one_y_train[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if one_y_train[i]!=final_class[0] and ini_class[0]==one_y_train[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/5678.0, count_adv_pca/5678.0, count_pure_adv_pca/5678.0

In [ ]:
count_pure_adv_pca=0.0
count_adv_pca=0.0
count_wrong_pca=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(10000):
    x_ini=(X_test_dr[i,:]).reshape((1,rd))
    ini_class=clf_pca.predict(x_ini)
    #x_adv=(x_ini-0.25*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    #x_adv=X_adv_dr[i,:].reshape((1,rd))
    x_adv=X_adv_test_dr[i,:].reshape((1,rd))
    final_class=clf_pca.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_pca=count_adv_pca+1
    if y_test[i]!=final_class[0]:
        count_wrong_pca=count_wrong_pca+1
    if y_test[i]==ini_class[0]:
        count_correct=count_correct+1
    if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
        count_pure_adv_pca=count_pure_adv_pca+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_pca/10000.0, count_adv_pca/10000.0, count_pure_adv_pca/count_correct
print count_correct

one_decision_adv_dr=clf_pca.decision_function(one_X_adv_dr)

np.amax(one_decision_adv_dr[:,1])

np.amin(one_decision_adv_dr[:,1])

temp_gap_adv_dr=np.where(one_decision_adv_dr[:,1]>0)[0]

temp_gap_adv_dr

for i in range(10):
    print np.linalg.norm(clf_pca.coef_[i,:])

reverse_transform=pca.inverse_transform(X_train_dr[49999,:].reshape((1,rd)))

plt.imshow((reverse_transform.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

rev=pca.inverse_transform(x_ini)

plt.imshow((rev.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

rev_ad=pca.inverse_transform(x_adv)

plt.imshow((rev_ad.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

rev_ad_2=pca.inverse_transform(x_adv)

plt.imshow((rev_ad_2.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

### Random projections

In [ ]:
from sklearn import random_projection

In [ ]:
rd_list=[10,20,30,40,50,100,331,784]

In [ ]:
for rd in rd_list:
    rand_transform=random_projection.GaussianRandomProjection(n_components=rd,random_state=1)
    X_train_rand_dr=rand_transform.fit_transform(X_train)
    clf_rand_dr=svm.LinearSVC(dual=False)
    clf_rand_dr.fit(X_train_rand_dr,y_train)
    X_val_rand_dr=rand_transform.transform(X_val)
    X_test_rand_dr=rand_transform.transform(X_test)
#     X_adv_train_rand=rand_transform.transform(adv_examples_train)
#     X_adv_test_rand=rand_transform.transform(adv_examples_test)
    val_score=clf_rand_dr.score(X_val_rand_dr,y_val)
    test_score=clf_rand_dr.score(X_test_rand_dr,y_test)
    resultfile=open('SVM_rand_results.txt','a')
    resultfile.write(str(rd)+', recons: '+str.format("{0:.3f}",val_score)
                        +','+str.format("{0:.3f}",test_score)+'\n')
    counter=0
    for DEV_MAG in np.linspace(0.1,1.0,10):
        X_adv_test_rand=rand_transform.transform(adv_examples_test[:,:,counter])
        count_pure_adv_rand_dr=0.0
        count_adv_rand_dr=0.0
        count_wrong_rand_dr=0.0
        count_correct=0.0
        #print_flag_pca=0
        for i in range(10000):
            x_ini=(X_test_rand_dr[i,:]).reshape((1,rd))
            ini_class=clf_rand_dr.predict(x_ini)
            #x_adv=(x_ini-1.0*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[0,:])))).reshape((1,rd))
            x_adv=X_adv_test_rand[i,:].reshape((1,rd))
            final_class=clf_rand_dr.predict(x_adv)
            if ini_class[0]!=final_class[0]:
                count_adv_rand_dr=count_adv_rand_dr+1
            if y_test[i]!=final_class[0]:
                count_wrong_rand_dr=count_wrong_rand_dr+1
            if y_test[i]==ini_class:
                count_correct=count_correct+1
            if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
                count_pure_adv_rand_dr=count_pure_adv_rand_dr+1
        plotfile=open('Gradient_attack_SVM_rand_retrain'+str(rd)+'.txt','a')
        # print("Deviation {} took {:.3f}s".format(
        #     DEV_MAG, time.time() - start_time))
        plotfile.write(str(rd)+","+str(DEV_MAG)+","+
                        str(count_wrong_rand_dr/10000.0*100.0)+","+
                        #str.format("{0:.3f}",conf_wrong/count_tot)+","+
                        str(count_adv_rand_dr/10000.0*100.0)+","+
                        #str.format("{0:.3f}",conf_adv/count_tot)+","+
                        str.format("{0:.3f}",count_pure_adv_rand_dr/count_correct*100.0)+","+
                        #str.format("{0:.3f}",conf_abs/count_tot)+","+
#                         str.format("{0:.3f}",rd_perturb/50000.0)+","+
                        str(0)+"\n")
        plotfile.close()
        counter=counter+1
print count_wrong_rand_dr/10000.0, count_adv_rand_dr/10000.0, count_pure_adv_rand_dr/count_correct

In [ ]:
  count_pure_adv_rand_dr=0.0
        count_adv_rand_dr=0.0
        count_wrong_rand_dr=0.0
        count_correct=0.0
        #print_flag_pca=0
        for i in range(50000):
            x_ini=(X_train_rand_dr[i,:]).reshape((1,rd))
            ini_class=clf_rand_dr.predict(x_ini)
            #x_adv=(x_ini-1.0*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[0,:])))).reshape((1,rd))
            x_adv=X_adv_train_rand[i,:].reshape((1,rd))
            final_class=clf_rand_dr.predict(x_adv)
            if ini_class[0]!=final_class[0]:
                count_adv_rand_dr=count_adv_rand_dr+1
            if y_train[i]!=final_class[0]:
                count_wrong_rand_dr=count_wrong_rand_dr+1
            if y_train[i]==ini_class:
                count_correct=count_correct+1
            if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
                count_pure_adv_rand_dr=count_pure_adv_rand_dr+1
            #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
                #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
                #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
                #print_flag=print_flag+1
print count_wrong_rand_dr/50000.0, count_adv_rand_dr/50000.0, count_pure_adv_rand_dr/count_correct
print count_correct

In [ ]:
DEV_MAG_rd=1.0

In [ ]:
count_pure_adv_rand_dr=0.0
count_adv_rand_dr=0.0
count_wrong_rand_dr=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(50000):
    x_ini=(X_train_rand_dr[i,:]).reshape((1,rd))
    ini_class=clf_rand_dr.predict(x_ini)
    x_adv=(x_ini-DEV_MAG_rd*(clf_rand_dr.coef_[ini_class[0],:]/(np.linalg.norm(clf_rand_dr.coef_[0,:])))).reshape((1,rd))
    #x_adv=X_adv_train_rand[i,:].reshape((1,rd))
    final_class=clf_rand_dr.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_rand_dr=count_adv_rand_dr+1
    if y_train[i]!=final_class[0]:
        count_wrong_rand_dr=count_wrong_rand_dr+1
    if y_train[i]==ini_class:
        count_correct=count_correct+1
    if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
        count_pure_adv_rand_dr=count_pure_adv_rand_dr+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_rand_dr/50000.0, count_adv_rand_dr/50000.0, count_pure_adv_rand_dr/count_correct
print count_correct

In [ ]:
count_pure_adv_rand_dr=0.0
count_adv_rand_dr=0.0
count_wrong_rand_dr=0.0
count_correct=0.0
#print_flag_pca=0
for i in range(10000):
    x_ini=(X_test_rand_dr[i,:]).reshape((1,rd))
    ini_class=clf_rand_dr.predict(x_ini)
    x_adv=(x_ini-DEV_MAG_rd*(clf_rand_dr.coef_[ini_class[0],:]/(np.linalg.norm(clf_rand_dr.coef_[0,:])))).reshape((1,rd))
    #x_adv=X_adv_test_rand[i,:].reshape((1,rd))
    #x_adv=(x_ini-0.25*(clf_pca.coef_[ini_class[0],:]/(np.linalg.norm(clf_pca.coef_[ini_class[0],:])))).reshape((1,rd))
    final_class=clf_rand_dr.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv_rand_dr=count_adv_rand_dr+1
    if y_test[i]!=final_class[0]:
        count_wrong_rand_dr=count_wrong_rand_dr+1
    if y_test[i]==ini_class:
        count_correct=count_correct+1
    if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
        count_pure_adv_rand_dr=count_pure_adv_rand_dr+1
    #if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
        #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
        #print_flag=print_flag+1
print count_wrong_rand_dr/10000.0, count_adv_rand_dr/10000.0, count_pure_adv_rand_dr/count_correct
print count_correct

In [ ]:
for i in range(10):
    print np.linalg.norm(clf_rand_dr.coef_[i,:])

In [ ]:
rand_matrix=rand_transform.components_

In [ ]:
rand_matrix

In [ ]:
rand_matrix_T=np.transpose(rand_matrix)

In [ ]:
rand_matrix_T.shape

In [ ]:
rev_test_0=np.dot(rand_matrix_T,X_test_rand_dr[0,:])

In [ ]:
rev_test_0.shape

In [ ]:
plt.imshow((rev_test_0.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)

In [ ]:
rand_rand_T=np.dot(rand_matrix,rand_matrix_T)

In [ ]:
rand_rand_T.shape

In [ ]:
a=np.dot(rand_matrix,X_test[0,:])

In [ ]:
X_test_rand_dr[0,:]-a

In [ ]:
rev_test_rand=np.dot(rand_matrix_T,X_test_rand_dr.T)

In [ ]:
rev_test_rand.shape

In [ ]:
clf.score(rev_test_rand.T,y_test)

### Kernel PCA

In [ ]:
from sklearn.decomposition import KernelPCA

In [ ]:
kpca_poly=KernelPCA(n_components=50,kernel='poly')

In [ ]:
X_train_poly_dr=kpca_poly.fit_transform(X_train)